In [1]:
%load_ext autoreload
%autoreload 2

### Cache data - price, prododuct data etc
suppoedly it caches itself in data.py, but that logic broke

In [2]:
from data.data import PriceData, ProductData, BenchmarkData, START_DATE, Benchmarks

In [3]:
# .9s
benchmark_data = BenchmarkData()
tickers = benchmark_data.get_constituents(Benchmarks.SP500)
len(tickers)

Scraped 503 tickers for SP500 from Wikipedia
Cached 1 entries


503

In [4]:
# 1m20s
product_data = ProductData()
data = product_data.get_data(tickers=tickers[:10])
len(data)

10

In [5]:
# 32.4s
price_data = PriceData()
data = price_data.get_data(tickers[:10], start_date=START_DATE, end_date='2025-06-01')
len(data)

3

### Backtesting

In [2]:
# first create ConstraintsConfig and PortfolioConfig
from backtesting.backtest import Backtest
from backtesting.scenarios import Scenario
from portfolio.constraints import ConstraintsConfig
from portfolio.portfolio import CapitalGrowthFrequency, PortfolioConfig
from strategies.strategy import StrategyTypes
from data.data import Benchmarks
verbose = True

constraints_config = ConstraintsConfig(
    long_only=True,
    cash_pct=0.0,
    max_long_count=0.3,
    max_short_count=0.3,
    max_buy_size=0.3,
)
portfolio_config = PortfolioConfig(
    initial_capital=100_000,
    initial_holdings={},
    new_capital_growth_amt=10000,
    capital_growth_freq=CapitalGrowthFrequency.MONTHLY.value,
)

# then create scenario, note portfolio is created within scenario
scenario = Scenario(
    name="test backtest function",
    start_date="2020-01-01",
    end_date="2022-01-01",
    benchmark=Benchmarks.SP500,
    constraints=constraints_config,
    portfolio_config=portfolio_config,
    verbose=verbose,
)

# then create strategies
strategies = {StrategyTypes.MACD_CROSSOVER: True, StrategyTypes.RSI_CROSSOVER: True}
scenario.set_strategies(strategies)


# then create backtest
backtest = Backtest(scenario, verbose=True)

# then run test!
backtest.run_batch()

Portfolio setup: {
    "initial_capital": 100000,
    "initial_holdings": {},
    "initial_value": 0,
    "capital_growth_freq": "M",
    "new_capital_growth_pct": 0,
    "new_capital_growth_amt": 10000,
    "allocation_method": "equal",
    "exclude_sectors": [],
    "include_countries": [
        "United States"
    ],
    "min_market_cap": 0,
    "max_market_cap": Infinity
}
Strategies: {
    "macd_x": true,
    "rsi_x": true
}
Backtest starting... swoosh!
Universe size: 480
Total trading days: 505
Total strategies: 2
Starting in 2020-01-01
Ending in 2022-01-01


Backtesting by strategy: 100%|██████████| 2/2 [00:01<00:00,  1.36strategy/s]


Short trade amount 260 too large violates max trade constraint
Short trade amount 291 too large violates max trade constraint
Short trade amount 254 too large violates max trade constraint
Short trade amount 322 too large violates max trade constraint
Short trade amount 266 too large violates max trade constraint
Backtest completed!


In [3]:
backtest.generate_report(filename="test")

Report saved to: reports/test.pdf


'reports/test.pdf'

### Gridsearch

In [4]:
from backtesting.grid_search import GridSearch
from backtesting.scenarios import Scenario
from config import DEFAULT_CONSTRAINTS, DEFAULT_PORTFOLIO_SETUP, Benchmarks
from strategies.strategy import StrategyTypes

# first create grid search params
grid_search_params =[
    {StrategyTypes.MACD_CROSSOVER: True},
    {StrategyTypes.MACD_CROSSOVER: True, StrategyTypes.BOLLINGER_BANDS: False},
    {StrategyTypes.RSI_CROSSOVER: True, StrategyTypes.Z_SCORE_MEAN_REVERSION: True},
    {
        StrategyTypes.MACD_CROSSOVER: True,
        StrategyTypes.RSI_CROSSOVER: True,
        StrategyTypes.BOLLINGER_BANDS: True,
        StrategyTypes.Z_SCORE_MEAN_REVERSION: True,
    },
]

# then create scenario
scenario = Scenario(
    name="test grid search function",
    start_date="2020-01-01",
    end_date="2022-01-01",    
    constraints=DEFAULT_CONSTRAINTS,
    portfolio_config=DEFAULT_PORTFOLIO_SETUP,
    benchmark=Benchmarks.SP500,
    verbose=False,
)

# then create grid search
gs = GridSearch(base_scenario=scenario, max_workers=10, verbose=True)
gs.set_grid_params(grid_search_params)

# then run grid search
results = gs.run()

# can print the run schedule
print(gs.get_grid_search_schedule())

# then print results
print(gs.results_to_dataframe())

Running grid search with 4 parameter combinations...


Grid search progress:   0%|          | 0/4 [00:00<?, ?it/s]

Short trade amount 247 too large violates max trade constraint
Short trade amount 253 too large violates max trade constraint
Short trade amount 369 too large violates max trade constraint
Short trade amount 350 too large violates max trade constraint
Short trade amount 345 too large violates max trade constraint
Short trade amount 244 too large violates max trade constraint
Short trade amount 429 too large violates max trade constraint
Short trade amount 369 too large violates max trade constraint
Short trade amount 249 too large violates max trade constraint
Short trade amount 249 too large violates max trade constraint
Short trade amount 290 too large violates max trade constraint
Long trade amount 243 too large violates max trade constraint
Long trade amount 266 too large violates max trade constraint


Grid search progress:  25%|██▌       | 1/4 [00:01<00:04,  1.41s/it]

Short trade amount 261 too large violates max trade constraint
Short trade amount 348 too large violates max trade constraint
Short trade amount 313 too large violates max trade constraint
Short trade amount 323 too large violates max trade constraint
Short trade amount 381 too large violates max trade constraint
Short trade amount 306 too large violates max trade constraint
Short trade amount 260 too large violates max trade constraint
Short trade amount 291 too large violates max trade constraint
Short trade amount 254 too large violates max trade constraint
Short trade amount 322 too large violates max trade constraint
Short trade amount 266 too large violates max trade constraint


Grid search progress: 100%|██████████| 4/4 [00:03<00:00,  1.30it/s]


Grid search completed! Found 4 valid results.
  grid_num                              param_name
1        1                           **Pos: macd_x
2        2            **Pos: macd_x **Neg: b_bands
0        3                       **Pos: rsi_x || z
3        4  **Pos: macd_x || rsi_x || b_bands || z
  grid_num                              param_name  total_return  \
1        1                           **Pos: macd_x      2.837402   
2        2            **Pos: macd_x **Neg: b_bands      3.185413   
0        3                       **Pos: rsi_x || z      1.651897   
3        4  **Pos: macd_x || rsi_x || b_bands || z      1.140308   

   annualized_return  annualized_sharpe  annualized_ir  \
1           0.958929                NaN            NaN   
2           1.060554           1.001434     -12.400727   
0           0.651312           0.810314     -32.051398   
3           0.468567           1.140848      -8.479076   

   average_holding_period  max_holding_amount  
1              228.